In [23]:
!pip install tokenizers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 160.2 kB/s eta 0:00:0000:0100:02


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import random
import os
import re

from tokenizers import Tokenizer
from tokenizers.models import BPE, WordPiece, Unigram
from tokenizers.trainers import BpeTrainer, WordPieceTrainer, UnigramTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.normalizers import Lowercase
from tokenizers.decoders import WordPiece as WordPieceDecoder


In [2]:
! tail -n 8 data/anek.txt

<|startoftext|>Если бесконечное количество российских футболистов запустить на бесконечное количество футбольных полей и дать им бесконечное количество времени, то один из них когда-нибудь забьёт гол.

<|startoftext|>На чемпионат мира по футболу от России нужно Юлию Самойлову отправлять, хоть какая-то надежда на победу будет.

<|startoftext|>В целях профилактики от всего весной следует есть много чеснока. От женщин, кстати, тоже помогает.

<|startoftext|>На моих глазах как-то две девушки затаскивали кавказца в машину. Они худенькие, а он здоровый такой, никак не хотел в машину лезть. Они попросили у меня помощи, сказали, что собаку надо в ветклинику отвезти.



In [2]:
with open('data/anek.txt', 'r') as f:
    aneki = f.read().strip().replace('<|startoftext|>', '').split('\n\n')

In [18]:
aneki[0:5]

['Только заметил, что слово "п@рно" набирается самими центральными клавишами. Как все продумано, блин!',
 'Друзья мои, чтобы соответствовать вам, я готов сделать над собой усилие и стать лучше. Но тогда и вы станьте немного хуже!',
 '- Люся, ты все еще хранишь мой подарок?- Да.- Я думал, ты выкинула все, что со мной связано.- Плюшевый мишка не виноват, что ты ебл@н...',
 '- А вот скажи честно, ты во сне храпишь?- Понятие не имею, вроде, нет. От собственного храпа по крайней мере еще ни разу не просыпался.- Ну, так у жены спроси.- А жена и подавно не знает. У нее странная привычка после замужества возникла: как спать ложится - беруши вставляет.',
 'Поссорилась с мужем. Пока он спал, я мысленно развелась с ним, поделила имущество, переехала, поняла, что жить без него не могу, дала последний шанс, вернулась. В итоге, ложусь спать уже счастливой женщиной.']

In [3]:
# слово отделяется от другого любой последовательностью пробелов или пунктуации
regex = re.compile('\\w+|[^\\w\\s]+')

vocabulary = set()

for anek in aneki:
    vocabulary |= set(regex.findall(anek))

print('Число уникальных слов:', len(vocabulary))
# Output
# Число уникальных слов: 159497


Число уникальных слов: 159497


In [4]:
class WhitespaceTokenizer:
    def __init__(self, corpus, vocab_size=-1, n=2):
        self.n = n
        self.regex = re.compile('\\w+|[^\\w\\s]+')

        words = []
        for line in corpus:
            # разбиваем по пробелам
            words.extend(self.regex.findall(line))
            
        # считаем встречаемость слов
        word_count = Counter(words)
        if vocab_size == -1:  # если словарь не ограничен
            self.vocab = set(word_count)
        else:
            common = word_count.most_common(vocab_size - 3)  # 3 специальных токена
            self.vocab, _ = zip(*common)
            self.vocab = set(self.vocab)

        self.vocab |= set(['[UNK]', '[BOS]', '[EOS]'])

    def encode(self, text):
        # токенизируем
        words = self.regex.findall(text)
        encoded = [w if w in self.vocab else '[UNK]' for w in words]
        # дополняем последовательность специальными токенами
        encoded = ['[BOS]'] * self.n + encoded + ['[EOS]']
        return WhitespaceOutput(encoded)

    def get_vocab(self):
        return self.vocab


In [5]:
class WhitespaceOutput:
    def __init__(self, tokens):
        self.tokens = tokens
        self.ids = None


def get_tokenizer(corpus, tokenizer_type, vocab_size=32768, n=2, lowercase=False):
    assert tokenizer_type in ['whitespace', 'bpe', 'wordpiece', 'unigram']
    
    if tokenizer_type == 'whitespace':
        return WhitespaceTokenizer(corpus, vocab_size=vocab_size, n=n)
    if tokenizer_type == 'bpe':
        model_class = BPE
        trainer_class = BpeTrainer
    elif tokenizer_type == 'wordpiece':
        model_class = WordPiece
        trainer_class = WordPieceTrainer
    elif tokenizer_type == 'unigram':
        model_class = Unigram
        trainer_class = UnigramTrainer

    if tokenizer_type == 'unigram':
        # unk_token передается в Trainer
        tokenizer = Tokenizer(model_class())
    else:
        tokenizer = Tokenizer(model_class(unk_token='[UNK]'))
        # делим текст по пробелам (а так же символам пунктуации)
        tokenizer.pre_tokenizer = Whitespace()

    if lowercase:
        # если надо, приводим текст к нижнему регистру
        tokenizer.normalizer = Lowercase()

    # указываем параметры токенизатора
    trainer = trainer_class(
        vocab_size=vocab_size,
        # спецсимвол для указания, что токен является продолжением другого
        continuing_subword_prefix='##',
        special_tokens=['[BOS]', '[EOS]', '[UNK]', '[PAD]'],
        unk_token='[UNK]'
    )

    # формируем файл с текстами
    filename = "%08x" % random.getrandbits(32)
    with open(filename, 'w') as f:
        for line in corpus:
            f.write(f'{line}\\n')

    tokenizer.train([filename], trainer)
    os.remove(filename)
    
    if tokenizer_type != 'unigram':
        # декодер, который склеивает токен с предыдущим, если в начале стоит ##
        tokenizer.decoder = WordPieceDecoder(prefix='##', cleanup=True)
    else:
        # склеиваем все токены
        tokenizer.decoder = WordPieceDecoder(prefix='', cleanup=True)

    return tokenizer


In [ ]:
tokenizer = get_tokenizer(aneki, 'bpe')

Ignored unknown kwargs option unk_token





In [27]:
output = tokenizer.encode('Кто сказал, что солдат мечтает стать генералом? Солдат мечтает стать хлеборезом.')
print(output.tokens)
print(output.ids)

# Output
# ['Кто', 'сказал', ',', 'что', 'солдат', 'мечтает', 'стать', 'генералом', '?', 'Солдат', 'мечтает', 'стать', 'хлебо', '##рез', '##ом', '.']
# [1333, 1232, 13, 422, 5377, 7868, 1873, 20185, 31, 12911, 7868, 1873, 31022, 1981, 625, 15]


['Кто', 'сказал', ',', 'что', 'солдат', 'мечтает', 'стать', 'генералом', '?', 'Солдат', 'мечтает', 'стать', 'хлебо', '##рез', '##ом', '.']
[1613, 1257, 13, 428, 5452, 7973, 1903, 20451, 31, 26223, 7973, 1903, 31628, 1982, 617, 15]


In [28]:
tokenizer.decode(output.ids)

# Output
# 'Кто сказал, что солдат мечтает стать генералом? Солдат мечтает стать хлеборезом.'


'Кто сказал, что солдат мечтает стать генералом? Солдат мечтает стать хлеборезом.'

In [29]:
from tokenizers.processors import TemplateProcessing

def get_post_processor(n=2):
    # post_processor добавит специальные токены в начало и конец последовательности
    post_processor = TemplateProcessing(
        single='[BOS] ' * n + '$A [EOS]',
        special_tokens=[
            ("[BOS]", tokenizer.token_to_id("[BOS]")),
            ("[EOS]", tokenizer.token_to_id("[EOS]")),
        ],
    )
    return post_processor


In [30]:
tokenizer.post_processor = get_post_processor(n=2)

output = tokenizer.encode('Кто сказал, что солдат мечтает стать генералом? Солдат мечтает стать хлеборезом.')
print(output.tokens)
print(output.ids)

# Output
# ['[BOS]', '[BOS]', 'Кто', 'сказал', ',', 'что', 'солдат', 'мечтает', 'стать', 'генералом', '?', 'Солдат', 'мечтает', 'стать', 'хлебо', '##рез', '##ом', '.', '[EOS]']
# [0, 0, 1333, 1232, 13, 422, 5377, 7868, 1873, 20185, 31, 12911, 7868, 1873, 31022, 1981, 625, 15, 1]


['[BOS]', '[BOS]', 'Кто', 'сказал', ',', 'что', 'солдат', 'мечтает', 'стать', 'генералом', '?', 'Солдат', 'мечтает', 'стать', 'хлебо', '##рез', '##ом', '.', '[EOS]']
[0, 0, 1613, 1257, 13, 428, 5452, 7973, 1903, 20451, 31, 26223, 7973, 1903, 31628, 1982, 617, 15, 1]


In [31]:
from collections import defaultdict, Counter

def count_ngrams(aneki, tokenizer, n=2):
    counts = defaultdict(Counter)

    for anek in tqdm(aneki):
        tokens = tokenizer.encode(anek).tokens

        for i in range(n, len(tokens)):
            prefix = tuple(tokens[i - n:i])  # префикс (n токенов)
            token = tokens[i]  # последний токен
            counts[prefix][token] += 1

    return counts


In [32]:
dummy_lines = aneki[-4:]
dummy_lines


['Если бесконечное количество российских футболистов запустить на бесконечное количество футбольных полей и дать им бесконечное количество времени, то один из них когда-нибудь забьёт гол.',
 'На чемпионат мира по футболу от России нужно Юлию Самойлову отправлять, хоть какая-то надежда на победу будет.',
 'В целях профилактики от всего весной следует есть много чеснока. От женщин, кстати, тоже помогает.',
 'На моих глазах как-то две девушки затаскивали кавказца в машину. Они худенькие, а он здоровый такой, никак не хотел в машину лезть. Они попросили у меня помощи, сказали, что собаку надо в ветклинику отвезти.']

In [33]:
dummy_counts = count_ngrams(dummy_lines, tokenizer, n=2)

print(dummy_counts[('[BOS]', '[BOS]')])
print(dummy_counts[('в', 'машину')])

# Output
# Counter({'На': 2, 'Если': 1, 'В': 1})
# Counter({'.': 1, 'лезть': 1})


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/4 [00:00<?, ?it/s]

Counter({'На': 2, 'Если': 1, 'В': 1})
Counter({'.': 1, 'лезть': 1})


In [9]:
from typing import Union

class NGramLanguageModel:
    def __init__(self, corpus, tokenizer, n=2):
        self.tokenizer = tokenizer
        self.tokenizer.post_processor = get_post_processor(n=n)
        
        self.counts = count_ngrams(corpus, tokenizer, n=n)
        self.n = n

        # посчитаем, сколько раз встречается каждый префикс
        self.prefix_counts = dict()
        for prefix, token_count in self.counts.items():
            self.prefix_counts[prefix] = sum(token_count.values())

    def get_last_n_tokens(self, prefix: Union[list, str]):
        if isinstance(prefix, str):
            prefix = self.tokenizer.encode(prefix).tokens[:-1]  # последний токен – [EOS]

        return tuple(prefix[-self.n:])

    def get_next_tokens_and_probs(self, prefix: Union[list, str]):
        prefix = self.get_last_n_tokens(prefix)

        possible_tokens = self.counts[prefix]  # dict <token: count>
        
        # возвращаем символ конца строки, если нет подходящих продолжений
        if len(possible_tokens) == 0:
            return ['[EOS]'], [1]

        tokens = list(possible_tokens.keys())
        
        counts = list(possible_tokens.values())
        probs = np.array(counts) / self.prefix_counts[prefix]

        return tokens, probs

    def get_token_prob(self, token, prefix):
        prefix = self.get_last_n_tokens(prefix)
        
        possible_tokens = self.counts[prefix]  # dict <token: count>
        token_count = possible_tokens.get(token, 0)
        if token_count == 0:
            return 0
        else:
            return token_count / self.prefix_counts[prefix]


In [35]:
lm = NGramLanguageModel(aneki, tokenizer, n=3)


  0%|          | 0/124155 [00:00<?, ?it/s]

In [36]:
def get_next_token(lm, prefix):
    tokens, probs = lm.get_next_tokens_and_probs(prefix)

    next_token = np.random.choice(tokens, p=probs)
    return next_token


In [37]:
np.random.seed(100)

prefix = lm.tokenizer.encode('Приходит парень').tokens[:-1]  # последний токен – [EOS]

for i in range(100):
    prefix.append(get_next_token(lm, prefix))
    if prefix[-1] == '[EOS]':
        break

print('Токены:', prefix)
print()
print('Текст:', lm.tokenizer.decode([lm.tokenizer.token_to_id(p) for p in prefix]))


Токены: ['[BOS]', '[BOS]', '[BOS]', 'Приходит', 'парень', 'в', 'институт', ',', 'но', 'не', 'совсем', '.', 'Вы', '##ставил', 'бутылку', 'какого', '-', 'то', 'тай', '##ного', 'ор', '##дена', ',', 'который', 'бьется', 'за', 'то', ',', 'что', 'в', 'связи', 'с', 'провед', '##ением', 'ги', '##драв', '##ли', '##ческих', 'испыта', '##ний', 'она', 'бес', '##след', '##но', 'исчезают', 'вещи', '.', 'Видимо', ',', 'перем', '##ывают', 'косточки', 'птиц', 'улет', '##евших', 'зачем', '-', 'то', 'свою', 'игрушку', 'на', 'стол', 'и', 'при', '##п', '##уд', '##ривает', 'т', '##аль', '##ком', ',', 'говорит', 'ей', 'так', 'уча', '##стли', '##во', ':-', 'Женщина', ',', 'передайте', 'за', 'проезд', '!-', 'А', 'волшебное', 'слово', '?-', 'Б', '##егом', ',', 'твою', 'мать', '!', 'И', 'ведь', 'кто', '-', 'то', 'же', 'должен', 'держать', 'свечку', ',', 'чтобы']

Текст: Приходит парень в институт, но не совсем. Выставил бутылку какого - то тайного ордена, который бьется за то, что в связи с проведением гидравлич

In [6]:
def perplexity(lm, lines, min_prob=10 ** -50.):
    # min_prob нужен, чтобы не брать логарифм 0,
    # если в тестовой выборке есть новая n-грамма
    ppls = []
    for line in lines:
        tokens = lm.tokenizer.encode(line).tokens
        log_ppl = 0
        for i in range(lm.n, len(tokens)):
            log_ppl += np.log(max(
                min_prob,
                lm.get_token_prob(tokens[i], tokens[:i])
            ))
        text_len = len(tokens) - lm.n  # вычитаем число [BOS] токенов
        ppls.append(np.exp(-log_ppl / text_len))

    return np.mean(ppls)


In [39]:
lm1 = NGramLanguageModel(dummy_lines, tokenizer, n=1)
lm3 = NGramLanguageModel(dummy_lines, tokenizer, n=3)
lm10 = NGramLanguageModel(dummy_lines, tokenizer, n=10)

ppx1 = perplexity(lm1, dummy_lines)
ppx3 = perplexity(lm3, dummy_lines)
ppx10 = perplexity(lm10, dummy_lines)

print("Perplexities: ppx1=%.3f ppx3=%.3f ppx10=%.3f" % (ppx1, ppx3, ppx10))



  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

Perplexities: ppx1=1114253017947795.875 ppx3=2427167859425.817 ppx10=1.049


In [7]:
from sklearn.model_selection import train_test_split

train_lines, test_lines = train_test_split(aneki, test_size=0.25, random_state=42)


In [41]:
# строим токенизатор ТОЛЬКО на ОБУЧАЮЩЕЙ выборке
tokenizer = get_tokenizer(train_lines, 'bpe')


Ignored unknown kwargs option unk_token





In [42]:
for n in (1, 2, 3):
    lm = NGramLanguageModel(train_lines, tokenizer, n=n)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))


  0%|          | 0/93116 [00:00<?, ?it/s]

N = 1, Perplexity = 2611138789471461499620342605283328.00000


  0%|          | 0/93116 [00:00<?, ?it/s]

N = 2, Perplexity = 3554294196110188807941063757886144821195702272.00000


  0%|          | 0/93116 [00:00<?, ?it/s]

N = 3, Perplexity = 234308237974753647225500111887375138263626940416.00000


In [10]:
class SmoothNGramLM(NGramLanguageModel):
    def __init__(self, corpus, tokenizer, n=2, delta=1.0):
        super().__init__(corpus, tokenizer, n=n)
        self.delta = delta
        self.vocab = set(self.tokenizer.get_vocab())
        self.n = n

        self.smooth_probs = defaultdict(Counter)
        # посчитаем сглаженные вероятности
        for prefix, token_count in self.counts.items():
            prefix_count = sum(token_count.values())
            for token, count in token_count.items():
                self.smooth_probs[prefix][token] = (count + delta) / (prefix_count + len(self.vocab) * delta)

    def get_next_tokens_and_probs(self, prefix: Union[list, str]):
        prefix = super().get_last_n_tokens(prefix)

        token_probs = self.smooth_probs[prefix]  # dict <token: smooth_prob>

        tokens = list(token_probs.keys())
        probs = list(token_probs.values())
        
        # перераспределям оставшуюся вероятность
        excess_prob = 1.0 - sum(probs)
        unseen_tokens = self.vocab - set(tokens)
        unseen_prob = excess_prob / (len(unseen_tokens) + 1e-6)  # деление на 0, если n=1

        smooth_tokens = tokens + list(unseen_tokens)
        smooth_probs = np.hstack((
            probs,
            np.full(len(unseen_tokens), fill_value=unseen_prob)  # массив одинаковых вероятностей
        ))
        return smooth_tokens, smooth_probs

    def get_token_prob(self, token, prefix):        
        prefix = super().get_last_n_tokens(prefix)
        
        token_probs = self.smooth_probs[prefix]
        
        prob = token_probs.get(token, 0)
        if prob > 0:  # знаем вероятность для токена
            return prob
        else:  # не знаем вероятность для токена
            excess_prob = 1.0 - sum(token_probs.values())
            n_unseen_tokens = len(self.vocab) - len(token_probs)
            return excess_prob / (n_unseen_tokens + 1e-6)


In [44]:
dummy_tokenizer = get_tokenizer(dummy_lines, 'bpe', vocab_size=128)


Ignored unknown kwargs option unk_token





In [45]:
for n in (0, 1, 2, 3):
    dummy_lm = SmoothNGramLM(dummy_lines, dummy_tokenizer, n=n)
    token_probs = []
    for token in dummy_lm.vocab:
        token_probs.append(dummy_lm.get_token_prob(token, ['в']))

    assert np.allclose(sum(token_probs), 1)


  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

In [46]:
for n in (0, 1, 2, 3):
    lm = SmoothNGramLM(train_lines, tokenizer=tokenizer, n=n, delta=0.1)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))


  0%|          | 0/93116 [00:00<?, ?it/s]

N = 0, Perplexity = 29278.39106


  0%|          | 0/93116 [00:00<?, ?it/s]

N = 1, Perplexity = 1410.09250


  0%|          | 0/93116 [00:00<?, ?it/s]

N = 2, Perplexity = 8414.21721


  0%|          | 0/93116 [00:00<?, ?it/s]

N = 3, Perplexity = 16523.13916


In [47]:
for n in (0, 1, 2, 3):
    lm = SmoothNGramLM(train_lines, tokenizer=tokenizer, n=n, delta=0.01)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))


  0%|          | 0/93116 [00:00<?, ?it/s]

N = 0, Perplexity = 29278.29256


  0%|          | 0/93116 [00:00<?, ?it/s]

N = 1, Perplexity = 822.42003


  0%|          | 0/93116 [00:00<?, ?it/s]

N = 2, Perplexity = 5399.91270


  0%|          | 0/93116 [00:00<?, ?it/s]

N = 3, Perplexity = 13502.74112


In [48]:
def unseen_n_grams(lm, test_corpus):
    number_unseen_n_grams = 0
    
    for line in test_corpus:
        tokens = tuple(lm.tokenizer.encode(line).tokens[:-1])
        
        for i in range(lm.n, len(tokens)):
            prefix = tuple(tokens[i - lm.n:i])
            if len(lm.counts[prefix]) == 0:
                number_unseen_n_grams += 1

    return number_unseen_n_grams


In [49]:
for n in (0, 1, 2, 3):
    lm = SmoothNGramLM(train_lines, tokenizer=tokenizer, n=n, delta=0.01)
    ppl = perplexity(lm, test_lines)
    unseen = unseen_n_grams(lm, test_lines)
    print("N = %i, Perplexity = %.2f, Unseen n-grams = %i" % (n, ppl, unseen))


  0%|          | 0/93116 [00:00<?, ?it/s]

N = 0, Perplexity = 29278.29, Unseen n-grams = 0


  0%|          | 0/93116 [00:00<?, ?it/s]

N = 1, Perplexity = 822.42, Unseen n-grams = 73


  0%|          | 0/93116 [00:00<?, ?it/s]

N = 2, Perplexity = 5399.91, Unseen n-grams = 189263


  0%|          | 0/93116 [00:00<?, ?it/s]

N = 3, Perplexity = 13502.74, Unseen n-grams = 487060


In [50]:
for n in (0, 1, 2, 3):
    whitespace_tokenizer = get_tokenizer(train_lines, 'whitespace', vocab_size=-1, n=n)
    lm = SmoothNGramLM(train_lines, tokenizer=whitespace_tokenizer, n=n, delta=0.01)
    ppl = perplexity(lm, test_lines)
    unseen = unseen_n_grams(lm, test_lines)
    
    print("N = %i, Perplexity = %.2f, Unseen n-grams = %.3f" % (n, ppl, unseen))


  0%|          | 0/93116 [00:00<?, ?it/s]

N = 0, Perplexity = 122302.09, Unseen n-grams = 0.000


  0%|          | 0/93116 [00:00<?, ?it/s]

N = 1, Perplexity = 4535.29, Unseen n-grams = 24232.000


  0%|          | 0/93116 [00:00<?, ?it/s]

N = 2, Perplexity = 28506.18, Unseen n-grams = 212554.000


  0%|          | 0/93116 [00:00<?, ?it/s]

N = 3, Perplexity = 61237.36, Unseen n-grams = 425518.000


In [51]:
for n in (0, 1, 2, 3):
    whitespace_tokenizer = get_tokenizer(train_lines, 'whitespace', vocab_size=32768, n=n)
    lm = SmoothNGramLM(train_lines, tokenizer=whitespace_tokenizer, n=n, delta=0.01)
    ppl = perplexity(lm, test_lines)
    unseen = unseen_n_grams(lm, test_lines)
    
    print("N = %i, Perplexity = %.2f, Unseen n-grams = %.3f" % (n, ppl, unseen))


  0%|          | 0/93116 [00:00<?, ?it/s]

N = 0, Perplexity = 27970.53, Unseen n-grams = 0.000


  0%|          | 0/93116 [00:00<?, ?it/s]

N = 1, Perplexity = 546.18, Unseen n-grams = 0.000


  0%|          | 0/93116 [00:00<?, ?it/s]

N = 2, Perplexity = 3374.95, Unseen n-grams = 124445.000


  0%|          | 0/93116 [00:00<?, ?it/s]

N = 3, Perplexity = 10177.99, Unseen n-grams = 347710.000


In [ ]:
tokenizer = get_tokenizer(train_lines, 'wordpiece')


Ignored unknown kwargs option unk_token





In [53]:
for n in (0, 1, 2, 3):
    lm = SmoothNGramLM(train_lines, tokenizer=tokenizer, n=n, delta=0.01)
    ppl = perplexity(lm, test_lines)
    unseen = unseen_n_grams(lm, test_lines)
    print("N = %i, Perplexity = %.2f, Unseen n-grams = %i" % (n, ppl, unseen))


  0%|          | 0/93116 [00:00<?, ?it/s]

N = 0, Perplexity = 29419.16, Unseen n-grams = 0


  0%|          | 0/93116 [00:00<?, ?it/s]

N = 1, Perplexity = 824.35, Unseen n-grams = 144


  0%|          | 0/93116 [00:00<?, ?it/s]

N = 2, Perplexity = 5361.20, Unseen n-grams = 189286


  0%|          | 0/93116 [00:00<?, ?it/s]

N = 3, Perplexity = 13535.41, Unseen n-grams = 484805


In [2]:
tokenizer = get_tokenizer(train_lines, 'unigram')


NameError: name 'get_tokenizer' is not defined